<a href="https://colab.research.google.com/github/SaifAlmaliki/chatbot-nltk/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade pip

Requirement already up-to-date: pip in /usr/local/lib/python3.6/dist-packages (20.2.3)


In [ ]:
import nltk
from nltk.stem import  WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

In [ ]:
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

**Import and load the data file**

The data file is in JSON format (intents.json) so we used the json package to parse the JSON file into Python.

In [ ]:
words = []
documents = []
classes = []
ignore_words = ['?', '!']

# Load json file
data_file = open('intents.json').read()
intents = json.loads(data_file)

**Preprocess data**

When working with text data, we need to perform various preprocessing on the data before we make a machine learning or a deep learning model. 
Tokenizing is the most basic and first thing you can do on text data. 

Tokenizing is the process of breaking the whole text into small parts like words.

Here we iterate through the patterns and tokenize the sentence using nltk.word_tokenize() function and append each word in the words list. We also create a list of classes for our tags.

In [ ]:
# iterate through the patterns and tokenize the sentence using nltk.word_tokenize()
nltk.download('punkt')

for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word
        w = nltk.word_tokenize(pattern)
        words.extend(w)

        # add documents in the corpus
        documents.append((w, intent['tag']))

        #create a list of classes for our tags.
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Now we will lemmatize each word and remove duplicate words from the list. 

Lemmatizing is the process of converting a word into its lemma form and then creating a pickle file to store the Python objects which we will use while predicting.

In [ ]:
nltk.download('wordnet')

#  lemmatize each word and remove duplicate words from the list
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]

# sort words
words = sorted(list(set(words)))

# sort classes
classes = sorted(list(set(classes)))

# documents = combination between patterns and intents
print(len(documents), 'documents')

# classes = intents
print(len(classes), 'classes', classes)

# words = all words, vocabulary
print(len(words), 'uniqe lemmatized words', words)

# creating a pickle file to store the Python objects which we will use while predicting.
pickle.dump(words, open('words.pkl', 'wb'))
pickle.dump(classes, open('classes.pkl', 'wb'))

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
47 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 uniqe lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 't

**Create training and testing data**

Now, we will create the training data in which we will provide the input and the output. 

Our input will be the pattern and output will be the class our input pattern belongs to. But the computer doesn’t understand text so we will convert text into numbers.

In [ ]:
# create our training data
training = []

# create an empty array for our output
output_empty = [0] * len(classes)

# training set, bag of words for each sentence
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]

    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0) 
    
    # output is '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)

# create train and test lists. X - patterns, Y - intents
train_x = list(training[: , 0])
train_y = list(training[:, 1])

print("Training data created")
print("training data shape", training.shape)

Training data created
training data shape (47, 2)


**Build the model**

We have our training data ready, now we will build a deep neural network that has **3 layers.** 

We use the Keras sequential API for this. After training the model for **200 epochs**, we achieved 100% accuracy on our model. 

Let us save the model as ‘chatbot_model.h5’.

In [ ]:
# create 3 layers. First layer 128 neurons,
# second layer 64 neurons
# 3rd output layer contains number of neurons equal to number of intents to predict output intent with softmax


model = Sequential()
model.add(Dense(128, 
                input_shape=(len(train_x[0]),),
                activation='relu'))

model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))


# Compile model. Stochastic gradient descent with Nesterov accelerated gradient 
# gives good results for this model
sgd = SGD(lr = 0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer = sgd, metrics=['accuracy'])

#fitting and saving the model 
hist = model.fit(np.array(train_x), 
                 np.array(train_y), 
                 epochs=200, 
                 batch_size=5, 
                 verbose=1)
model.save('Chatbot_model.h5', hist)
print('Model created')

Epoch 1/200
10/10 [==============================] - 0s 2ms/step - loss: 2.1739 - accuracy: 0.1489
Epoch 2/200
10/10 [==============================] - 0s 1ms/step - loss: 2.1911 - accuracy: 0.1277
Epoch 3/200
10/10 [==============================] - 0s 1ms/step - loss: 2.0339 - accuracy: 0.2553
Epoch 4/200
10/10 [==============================] - 0s 2ms/step - loss: 1.9944 - accuracy: 0.3404
Epoch 5/200
10/10 [==============================] - 0s 1ms/step - loss: 1.8767 - accuracy: 0.4468
Epoch 6/200
10/10 [==============================] - 0s 1ms/step - loss: 1.7685 - accuracy: 0.3830
Epoch 7/200
10/10 [==============================] - 0s 1ms/step - loss: 1.7546 - accuracy: 0.4681
Epoch 8/200
10/10 [==============================] - 0s 1ms/step - loss: 1.6658 - accuracy: 0.4681
Epoch 9/200
10/10 [==============================] - 0s 1ms/step - loss: 1.3521 - accuracy: 0.6809
Epoch 10/200
10/10 [==============================] - 0s 2ms/step - loss: 1.3062 - accuracy: 0.6170
Epoch 11/